## 图片重复检测神经网络

In [1]:
import concurrent.futures
from scipy import misc
import numpy as np
import pickle
from PIL import Image
import requests
from io import StringIO
from io import BytesIO
import threading
import _thread
import codecs
import json
import multiprocessing
from multiprocessing import Process
from multiprocessing import Manager
from multiprocessing import Queue
import matplotlib.pyplot as plt
import tensorflow as tf

import sklearn
from sklearn import cross_validation
import random

/home/recsys/platform/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## 全局变量

In [2]:
tfrecords_file = "/home/recsys/hzwangjian1/data/train.tfrecords"
file_to_write = "/home/recsys/hzwangjian1/data/imagepath_pair_duplabel.data"
model_path = '/home/recsys/hzwangjian1/data/duplicate_model_test'

In [3]:
# 获取《图片一路径，图片二路径，标记》数据对
def load_data():
    file_to_write = "/home/recsys/hzwangjian1/data/imagepath_pair_duplabel.data"
    reader_handler = open(file_to_write, 'r')

    image_one_path_list = []
    image_two_path_list = []
    label_list = []

    count = 0
    for line in reader_handler:
        count = count + 1
        elems = line.split("\t")
        if len(elems) < 3:
            print("len(elems) < 3:" + line)
            continue
        image_one_path = elems[0].strip()
        image_two_path = elems[1].strip()
        label = int(elems[2].strip())
#         if label == 0:
#             label = -1

        image_one_path_list.append(image_one_path)
        image_two_path_list.append(image_two_path)
        label_list.append(label)


    print(len(image_one_path_list))
    print(len(image_two_path_list))
    print(len(label_list))
    return image_one_path_list, image_two_path_list, label_list

## map多线程、多进程

In [4]:
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
def creat_thumbnail(image_path):
    img = misc.imread(image_path)
    img_arr = np.asarray(img)
    return img_arr

def load_image_with_path_list(image_path_list):
#     pool = Pool()
    pool = ThreadPool(30)
    image_list = pool.map(creat_thumbnail, image_path_list)
    pool.close()
    pool.join()
    
    return image_list

In [5]:
tf.reset_default_graph()
# Parameters
start_learning_rate = 0.1
end_learning_rate = 0.005
decay_step = 10000

training_iters = 200
batch_size = 50
display_step = 10

# Network Parameters
image_width = 256
image_height = 256
image_channel = 3
n_input = image_width * image_height

# tf Graph input
with tf.name_scope('input_data') as scope:
    X1 = tf.placeholder(tf.float32, [None, image_width, image_height, image_channel], name='image_one')
    X2 = tf.placeholder(tf.float32, [None, image_width, image_height, image_channel], name='image_two')
    y = tf.placeholder(tf.float32, [None], name='label')
    keep_prob = tf.placeholder(tf.float32, shape=(), name='drop_out') #dropout (keep probability)
    global_step = tf.Variable(0, trainable=False)
    learning_rate = tf.train.polynomial_decay(start_learning_rate, global_step, decay_step, end_learning_rate,power=3, cycle=False)
    

# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

variables_dict = {
    "W_conv1":tf.Variable(tf.random_normal(shape=[11,11, image_channel,32]), name='weight'),
    "b_conv1":tf.Variable(tf.random_normal(shape=[1,32]), name='bias'),
    
    "W_conv2":tf.Variable(tf.random_normal(shape=[5,5,32,64]), name='weight'),
    "b_conv2":tf.Variable(tf.random_normal( shape=[1,64]), name='bias'),
    
    "W_conv3":tf.Variable(tf.random_normal(shape=[3,3,64,64]), name='weight'),
    "b_conv3":tf.Variable(tf.random_normal(shape=[1,64], name='bias')),
    
    "W_full":tf.Variable(tf.random_normal(shape=[8 * 8 * 64, 4096]), name='weight'),
    "b_full":tf.Variable(tf.random_normal(shape=[1, 4096]), 'bias'),
    
    "W_full_dropout":tf.Variable(tf.random_normal(shape=[4096, 1024]), name='weight'),
    "b_full_dropout":tf.Variable(tf.random_normal(shape=[1, 1024]), 'bias')
}

# Create model
def conv_net(x,dropout):
    with tf.name_scope('model') as scope:
        # Reshape input picture
        x = tf.reshape(x, shape=[-1, image_width, image_height, image_channel])

        # Convolution Layer

        # Max Pooling (down-sampling)
        with tf.name_scope('layer1') as scope:
            W_conv1 = variables_dict["W_conv1"]
            b_conv1 = variables_dict["b_conv1"]
            convOne = tf.nn.conv2d(x, W_conv1, strides=[1,1,1,1], padding="SAME")
            reluOne = tf.nn.relu(convOne + b_conv1)
            conv1 = tf.nn.max_pool(reluOne, ksize=[1,4,4,1],strides=[1,4,4,1],padding="SAME")

        # Convolution Layer
        with tf.name_scope('layer2') as scope:
            W_conv2 = variables_dict["W_conv2"]
            b_conv2 = variables_dict["b_conv2"]
            convTwo = tf.nn.conv2d(conv1, W_conv2, strides=[1,1,1,1], padding="SAME")
            reluTwo = tf.nn.relu(convTwo + b_conv2)
            conv2 = tf.nn.max_pool(reluTwo, ksize=[1,4,4,1], strides=[1,4,4,1],padding="SAME")

        with tf.name_scope('layer3') as scope:
            W_conv3 = variables_dict["W_conv3"]
            b_conv3 = variables_dict["b_conv3"]
            convThree = tf.nn.conv2d(conv2, W_conv3, strides=[1,1,1,1], padding="SAME")
            reluThree = tf.nn.relu(convThree + b_conv3)
            conv3 = tf.nn.max_pool(reluThree, ksize=[1,2,2,1], strides=[1,2,2,1],padding="SAME")
            
        # Fully connected layer
        # Reshape conv2 output to fit fully connected layer input
        with tf.name_scope('full_connect') as scope:
            W_full = variables_dict["W_full"]
            b_full = variables_dict["b_full"]
            input_flat=tf.reshape(conv3, shape=[-1, 8 * 8 * 64])
            fc1 = tf.nn.relu(tf.matmul(input_flat, W_full) + b_full) #relu函数使得最终的图片特征都大于等于0
#             fc1 = tf.matmul(input_flat, W_full) + b_full

        # Apply Dropout
        with tf.name_scope('dropout_layer') as scope:
            drop_out = tf.nn.dropout(fc1,keep_prob)

        with tf.name_scope('full_connect_dropout') as scope:
            W_full_dropout = variables_dict["W_full_dropout"]
            b_full_dropout = variables_dict["b_full_dropout"]
            full_connect_dropout = tf.matmul(drop_out, W_full_dropout) + b_full_dropout
        
        
#         return fc1
        return full_connect_dropout

with tf.name_scope('whole_model') as scope:
#     with tf.device('/gpu:0'):
    img_one_rep = conv_net(X1, dropout=keep_prob)
#     with tf.device('/gpu:1'):
    img_two_rep = conv_net(X2, dropout=keep_prob)
    
    epsilon = 1e-12

    normalized_one_rep = tf.nn.l2_normalize(img_one_rep, 1, name="normalized_one_rep")
    normalized_two_rep = tf.nn.l2_normalize(img_two_rep, 1, name="normalized_two_rep")
    
    cos_rep = tf.reduce_sum(tf.mul(normalized_one_rep, normalized_two_rep), reduction_indices=1)
    
with tf.name_scope('result') as scope:
    with tf.name_scope('norm_W'):
        norm_W1 = tf.sqrt(tf.reduce_sum(tf.mul(variables_dict["W_conv1"], variables_dict["W_conv1"])))
        norm_W2 = tf.sqrt(tf.reduce_sum(tf.mul(variables_dict["W_conv2"], variables_dict["W_conv2"])))
        norm_W3 = tf.sqrt(tf.reduce_sum(tf.mul(variables_dict["W_conv3"], variables_dict["W_conv3"])))
        norm_full = tf.sqrt(tf.reduce_sum(tf.mul(variables_dict["W_full"], variables_dict["W_full"])))
        norm_W = norm_W1 + norm_W2 + norm_W3 + norm_full
        
    cross_entropy_cnn = tf.nn.sigmoid_cross_entropy_with_logits(logits=cos_rep * 10, targets=y)
                                
    cost = tf.reduce_sum(cross_entropy_cnn, name='cost')
        
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost, global_step=global_step)
    
    cos_rep_gt_zero = tf.greater(cos_rep, 0)
    label_gt_zero = tf.greater(y, 0.5)
    correct_pred = tf.equal(cos_rep_gt_zero, label_gt_zero)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

## 主入口

In [6]:
# 获取《图片一，图片二，标记》数据对 ， label等于 0 或 +1
image_one_path_list, image_two_path_list, label_list = load_data()  


# 数据划分
img_one_train, img_one_test, img_two_train, img_two_test, label_train, label_test = cross_validation.train_test_split(image_one_path_list, image_two_path_list, label_list, test_size= 0.2)
print(len(img_one_test))
print(len(img_two_test))
print(len(label_test))

# 获取图片数据
# print("获取图片数据")
# images_one = load_image_with_path_list(img_one_train)
# print(len(images_one))

# images_two = load_image_with_path_list(img_two_train)
# print(len(images_two))

images_test_one = load_image_with_path_list(img_one_test)
images_test_two = load_image_with_path_list(img_two_test)

13701
13701
13701
2741
2741
2741


In [ ]:
# for k in range(100):
#     fig = plt.figure()
#     fig.add_subplot(1,2,1)
#     plt.imshow(images_test_one[k])
#     fig.add_subplot(1,2,2)
#     plt.imshow(images_test_two[k])
#     plt.show()
#     print( str(label_test[k]))

In [7]:
sess = tf.InteractiveSession()
random_a = random.random()
resize_width = 128

def get_image(image_path):  
    """Reads the jpg image from image_path. 
    Returns the image as a tf.float32 tensor 
    Args: 
        image_path: tf.string tensor 
    Reuturn: 
        the decoded jpeg image casted to float32 
    """  
    content = tf.read_file(image_path)
    tf_image = tf.image.decode_jpeg(content, channels=3)
    
    random_a = random.random()
    if random_a < 0.3:
        new_width = random.randint(resize_width, image_width)
        tf_image = tf.image.resize_images(tf_image, [new_width, new_width]) ## 缩小
        tf_image = tf.image.resize_image_with_crop_or_pad(tf_image, image_height, image_width) ## 填充
    
    random_a = random.random()
    if random_a < 0.3:
        tf_image = tf.image.random_flip_left_right(tf_image) ## 左右翻转
    
    random_a = random.random()
    if random_a < 0.3:
        tf_image = tf.image.random_brightness(tf_image, 0.2) ## 亮度调整
    
    random_a = random.random()
    if random_a < 0.3:
        random_dgree = random.randint(-30,30)  ## 旋转
        pil_image = tf_image.eval()
        rotated = misc.imrotate(pil_image, random_dgree)
        tf_image = tf.convert_to_tensor(np.array(rotated))
    
#     random_a = random.random()
#     if random_a < 0.3:   ## 平移
#         random_left = random.randint(1, 30)
#         random_up = random.randint(1, 30)
#         random_right = random.randint(1, 30)
#         random_down = random.randint(1, 30)
        
#         random_height = image_height - random_up - random_down
#         random_width = image_width - random_left -  random_right
        
#         tf_image = tf.image.crop_to_bounding_box(tf_image, random_up, random_left, random_height, random_width)
        
#         random_pad_left = random.randint(0, image_width - random_width)
#         random_pad_up = random.randint(0, image_height - random_height)
#         tf_image = tf.image.pad_to_bounding_box(tf_image, random_pad_up, random_pad_left, image_height, image_width)
        
    
    return tf_image

## RunnerQueue
train_input_queue = tf.train.slice_input_producer( [img_one_train, img_two_train, label_train], shuffle=True, capacity=10 * batch_size)  
img_one_queue = get_image(train_input_queue[0])
img_two_queue = get_image(train_input_queue[1])
label_queue = train_input_queue[2]

batch_img_one, batch_img_two, batch_label = tf.train.shuffle_batch([img_one_queue, img_two_queue, label_queue],\
                                                                   batch_size=batch_size,capacity =  10 + 10* batch_size,\
                                                                   min_after_dequeue = 10,num_threads=16,\
                                                                  shapes=[(image_width, image_height, image_channel),\
                                                                          (image_width, image_height, image_channel),()])

In [ ]:
cost_summary = tf.scalar_summary("cost", cost)
accuracy_summary = tf.scalar_summary("accuracy", accuracy)
norm_summary = tf.scalar_summary("norm_W", norm_W)
# Initializing the variables
init = tf.initialize_all_variables()

# config = tf.ConfigProto(log_device_placement=True)
# sess = tf.Session(config=config)
sess.run(tf.initialize_all_variables())


summary_op = tf.merge_summary([cost_summary, accuracy_summary, norm_summary])
summary_writer = tf.train.SummaryWriter('/home/recsys/hzwangjian1/tensorboard/test', graph_def=sess.graph)


coord = tf.train.Coordinator()  
threads = tf.train.start_queue_runners(sess=sess,coord=coord)  

saver = tf.train.Saver()

for i in range(decay_step):

    batch_img_one_val, batch_img_two_val, label = sess.run([batch_img_one, batch_img_two,batch_label])
#     print(batch_im g_one_val.shape)
#     print(label)
    
    if i% 50 == 0:
        summary_str,train_accuracy,cos_rep_result,learning_rate_val  = sess.run([summary_op, accuracy, cos_rep, learning_rate],\
                                            feed_dict={X1:batch_img_one_val, X2:batch_img_two_val, y:label, keep_prob:(1.0)})

        print(np.sum(label) * 1.0 / len(label))
        print ("--------------------------------------------------step "+ str(i) +", training accuracy :"+ str(train_accuracy))
        summary_writer.add_summary(summary_str, i)
        print("cos_rep_result:")
        print(cos_rep_result[0:10])
        print("learning_rate_val:" + str(learning_rate_val))

        if i > 3000:
            for temp_index in range(len(label)):
                if (label[temp_index] > 0.5 and cos_rep_result[temp_index] < 0) or (label[temp_index] < 0.5 and cos_rep_result[temp_index] > 0):
                    fig = plt.figure()
                    fig.add_subplot(1,2,1)
                    plt.imshow(batch_img_one_val[temp_index])
                    fig.add_subplot(1,2,2)
                    plt.imshow(batch_img_two_val[temp_index])
                    plt.show()
                    print(str(label[temp_index]) + "\t" + str(cos_rep_result[temp_index]))
            

#     cos_rep_result, cross_entropy_val, normalized_one_rep_result = sess.run([cos_rep, cross_entropy_cnn, normalized_one_rep],\
#                                                  feed_dict={X1:batch_img_one_val, X2:batch_img_two_val, y:label, keep_prob:1.0})
#     print("cos_rep_result:")
#     print(cos_rep_result[0:10])
    
#     print("cross_entropy_val:")
#     print(cross_entropy_val)
#     print("normalized_image_fea:")
#     print(normalized_one_rep_result[1])
    
    sess.run([optimizer], feed_dict={X1:batch_img_one_val, X2:batch_img_two_val, y:label, keep_prob:0.75})

#     fig = plt.figure()
#     fig.add_subplot(1,2,1)
#     plt.imshow(batch_img_one_val[1])
#     fig.add_subplot(1,2,2)
#     plt.imshow(batch_img_two_val[1])
#     plt.show()
    
print("training done")
coord.request_stop()  
coord.join(threads)  
summary_writer.close()

saver.save(sess, model_path)

print("===========================================TEST==============================================")
num_accuracy = 0
total_num = 0

# images_test_one = load_image_with_path_list(img_one_test)
# images_test_two = load_image_with_path_list(img_two_test)

print(len(label_test))
for i in range(20):

    batch_img_one_test = images_test_one[i * batch_size : (i+1) * batch_size]
    batch_img_two_test = images_test_two[i * batch_size : (i+1) * batch_size]
    batch_label_test = label_test[i*batch_size : (i+1) * batch_size]
    
    print("======================================================================================batch:" + str(i))
    
    test_accuracy,cos_rep_gt_zero_val  = sess.run([accuracy, cos_rep_gt_zero],\
                             feed_dict={X1:batch_img_one_test, X2:batch_img_two_test, y:batch_label_test, keep_prob:(1.0)})
    print("test_accuracy:" + str(test_accuracy))
    num_accuracy = num_accuracy + test_accuracy * batch_size
    total_num = total_num + batch_size
    
    if i<= 2:
        for k in range(batch_size):
            fig = plt.figure()
            fig.add_subplot(1,2,1)
            plt.imshow(batch_img_one_test[k])
            fig.add_subplot(1,2,2)
            plt.imshow(batch_img_two_test[k])
            plt.show()
            print(str(cos_rep_gt_zero_val[k]) + "\t" + str(batch_label_test[k]))


print("num_accuracy:" + str(num_accuracy))
print("total_num:" + str(total_num))
print("test_accuracy_total:" + str(num_accuracy * 1.0 / total_num))

sess.close()

0.5
--------------------------------------------------step 0, training accuracy :0.5
cos_rep_result:
[ 0.98843759  0.85184222  0.93701923  0.9496848   0.99485016  0.99713147
  0.90344638  0.98998165  0.95203692  0.92556989]
learning_rate_val:0.1


## 测试模型加载

In [ ]:
config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)
sess = tf.Session(config=config)
saver = tf.train.Saver()
saver.restore(sess, model_path)
print("load model succeed!")

print("===========================================TEST==============================================")
num_accuracy = 0
total_num = 0

# images_test_one = load_image_with_path_list(img_one_test)
# images_test_two = load_image_with_path_list(img_two_test)

print(len(label_test))
for i in range(20):

    batch_img_one_test = images_test_one[i * batch_size : (i+1) * batch_size]
    batch_img_two_test = images_test_two[i * batch_size : (i+1) * batch_size]
    batch_label_test = label_test[i*batch_size : (i+1) * batch_size]
    
    print("======================================================================================batch:" + str(i))
    
    test_accuracy,cos_rep_gt_zero_val  = sess.run([accuracy, cos_rep_gt_zero],\
                             feed_dict={X1:batch_img_one_test, X2:batch_img_two_test, y:batch_label_test, keep_prob:(1.0)})
    print("test_accuracy:" + str(test_accuracy))
    num_accuracy = num_accuracy + test_accuracy * batch_size
    total_num = total_num + batch_size
    
    if i<= 2:
        for k in range(batch_size):
            fig = plt.figure()
            fig.add_subplot(1,2,1)
            plt.imshow(batch_img_one_test[k])
            fig.add_subplot(1,2,2)
            plt.imshow(batch_img_two_test[k])
            plt.show()
            print(str(cos_rep_gt_zero_val[k]) + "\t" + str(batch_label_test[k]))


print("num_accuracy:" + str(num_accuracy))
print("total_num:" + str(total_num))
print("test_accuracy_total:" + str(num_accuracy * 1.0 / total_num))

sess.close()

## 测试模型加载并提取一幅图片的特征

In [ ]:
config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)
sess = tf.Session(config=config)
saver = tf.train.Saver()
saver.restore(sess, model_path)
print("load model succeed!")

print("===========================================TEST==============================================")
num_accuracy = 0
total_num = 0

# images_test_one = load_image_with_path_list(img_one_test)
# images_test_two = load_image_with_path_list(img_two_test)

print(len(label_test))
for i in range(20):

    batch_img_one_test = images_test_one[i * batch_size : (i+1) * batch_size]
    batch_img_two_test = images_test_two[i * batch_size : (i+1) * batch_size]
    batch_label_test = label_test[i*batch_size : (i+1) * batch_size]
    
    print("======================================================================================batch:" + str(i))
    
    test_accuracy,cos_rep_gt_zero_val  = sess.run([accuracy, cos_rep_gt_zero],\
                             feed_dict={X1:batch_img_one_test, X2:batch_img_two_test, y:batch_label_test, keep_prob:(1.0)})
    print("test_accuracy:" + str(test_accuracy))
    num_accuracy = num_accuracy + test_accuracy * batch_size
    total_num = total_num + batch_size
    
    if i<= 2:
        for k in range(batch_size):
            fig = plt.figure()
            fig.add_subplot(1,2,1)
            plt.imshow(batch_img_one_test[k])
            fig.add_subplot(1,2,2)
            plt.imshow(batch_img_two_test[k])
            plt.show()
            print(str(cos_rep_gt_zero_val[k]) + "\t" + str(batch_label_test[k]))


print("num_accuracy:" + str(num_accuracy))
print("total_num:" + str(total_num))
print("test_accuracy_total:" + str(num_accuracy * 1.0 / total_num))

sess.close()

In [ ]:
# print("===========================================TEST==============================================")
# num_accuracy = 0
# total_num = 0

# images_test_one = load_image_with_path_list(img_one_test)
# images_test_two = load_image_with_path_list(img_two_test)

# print(len(label_test))
# for i in range(80):

#     batch_img_one_test = images_test_one[i*batch_size : (i+1) * batch_size]
#     batch_img_two_test = images_test_two[i*batch_size : (i+1) * batch_size]
#     batch_label_test = label_test[i*batch_size : (i+1) * batch_size]
    
#     print("======================================================================================batch:" + str(i))
    
#     test_accuracy,cos_rep_gt_zero_val  = sess.run([accuracy, cos_rep_gt_zero],\
#                              feed_dict={X1:batch_img_one_test, X2:batch_img_two_test, y:batch_label_test, keep_prob:(1.0)})
#     print("test_accuracy:" + str(test_accuracy))
#     num_accuracy = num_accuracy + test_accuracy * batch_size
#     total_num = total_num + batch_size
    
#     for k in range(batch_size):
#         fig = plt.figure()
#         fig.add_subplot(1,2,1)
#         plt.imshow(batch_img_one_val[k])
#         fig.add_subplot(1,2,2)
#         plt.imshow(batch_img_two_val[k])
#         plt.show()
#         print(cos_rep_gt_zero_val[k])


# print("num_accuracy:" + str(num_accuracy))
# print("total_num:" + str(total_num))
# print("test_accuracy_total:" + str(num_accuracy * 1.0 / total_num))

In [ ]:
# del images_one
# del images_two

# index = 98
# print(images_one[index].shape)

# fig = plt.figure()
# fig.add_subplot(1,2,1)
# plt.imshow(images_one[index])
# fig.add_subplot(1,2,2)
# plt.imshow(images_two[index])
# plt.show()

# print(len(images_one[1:5]))

# print(label_train[index])

# index_test = 98
# fig = plt.figure()
# fig.add_subplot(1,2,1)
# plt.imshow(images_test_one[index_test])
# fig.add_subplot(1,2,2)
# plt.imshow(images_test_two[index_test])
# plt.show()

# print(label_test[index_test])


# print( random.randint(0,len(img_one_train) - batch_size))